B13 setup

In [13]:
# write analog input onto DSC110 toptica scan
import sys
sys.path.append("..")
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from LowLevelModules.GeneralFunctions import *
from LowLevelModules.NIdaqAPD import *
from collections import deque
from LowLevelModules.bristol_wavelength_meter_client import *
import visa
from pyvisa import util
from LowLevelModules.Instruments import DG645
DG645_address = 'GPIB0::15::INSTR'

import nidaqmx
import numpy as np
import time

bristolCOM=8

bristol521handle=CLOpenUSBSerialDevice(bristolCOM)
bristol521handle.CLSetLambdaUnits('nm')

scan_terminal = '/Dev1/ao2'
offset_voltage = 0

# scan the FSM until the desired position
task_toptica = nidaqmx.Task("Toptica scan")
output_volt = task_toptica.ao_channels.add_ao_voltage_chan(scan_terminal, 'Piezo scan', min_val=-5, max_val=5)
task_toptica.write(offset_voltage, auto_start=True, timeout=5) # write a single data

1

In [ ]:
# set DAQ counter clock output

In [ ]:
# setup DG645 - 637, 737, RF switch
# in us
# Note - there is about 85 ns delay from external trigger to T0 rising edge

time_per_point = 1e6 #us 
init_time = 1 # 637 nm
PLE_time = 50 # 737 nm
delay637 = 0
delay737 = 0
delay532 = 1.030

delay_init = delay637
delay_resonance = delay737

marginFront = 0.100 # allow the excitation laser to truely turn off, let the defect decays to its ground state
marginBack = 0.050
# AB - 532, CD - 737, EF - 637 digital modulation, GH - RF switch
BNC_532 = 1
ch1_532 = 2
ch2_532 = 3

BNC_737 = 2
ch1_737 = 4
ch2_737 = 5

BNC_637 = 3
ch1_637 = 6
ch2_637 = 7

BNC_switch = 4
ch1_switch = 8
ch2_switch = 9

BNC_init = BNC_637
ch1_init = ch1_637
ch2_init = ch2_637

BNC_resonance = BNC_737
ch1_resonance = ch1_737
ch2_resonance = ch2_737

darkTime = marginFront + PLE_time + marginBack # excitation laser off

TTL_delay_resonance = max(0, delay_init + marginFront - delay_resonance )
TTL_delay_init = max(0, delay_resonance - delay_init - marginFront)

initMargin = 0.100

cycleEndInitTime = max(initMargin, init_time - TTL_delay_init - delay_init)
cycle_time = TTL_delay_init + delay_init + darkTime + cycleEndInitTime

num_cycle = np.ceil(time_per_point/cycle_time)


# set DG645 to burst mode
dg_source = DG645(DG645_address)

dg_source.set_delay(ch1_init,0,TTL_delay_init)
dg_source.set_delay(ch2_init,ch1_init,darkTime)

dg_source.set_delay(ch1_resonance,0,TTL_delay_resonance)
dg_source.set_delay(ch2_resonance,ch1_resonance,PLE_time)

dg_source.set_delay(ch1_switch,0,TTL_delay_resonance+delay_resonance)
dg_source.set_delay(ch2_switch,ch1_switch,PLE_time)

dg_source.set_burst_mode(1)
dg_source.set_burst_period(cycle_time)
dg_source.set_burst_delay(0)
dg_source.set_burst_count(num_cycle)

In [16]:
# scan laser frequency in discrete steps
# software timed
voltage = np.linspace(-1,1,50)
for v in voltage:
    task_toptica.write(v, auto_start=True, timeout=5)   
    # output high on a digital channel
    # ...
    currentLambda = CLGetLambdaReading(bristol521handle)
    currentPower = CLGetPowerReading(bristol521handle)
    
# Hardware timed 
# construct a list of voltages to write
# generate an on board clock with the rate desired

In [12]:
# set instruments back to default state and close communication channels
task_toptica.write(0)
task_toptica.close()
dg_source.close()